## Create projects

In [1]:
from morpheus.project.types.Permissions import Permissions, Visibility
from morpheus.common.types import DateTime
from morpheus.common.types.event_sourcing.EventMetadata import EventMetadata
from morpheus.common.types.event_sourcing.EventEnvelope import EventEnvelope
from morpheus.project.types.Project import ProjectId, Project, Metadata, Name, Description
from morpheus.project.domain.events.ProjectEvents.ProjectEvents import ProjectCreatedEvent
from morpheus.project.infrastructure.event_sourcing.ProjectEventBus import project_event_bus
from morpheus.project.types.User import UserId

user_id_1 = UserId.new()
user_id_2 = UserId.new()
project_user_1_ids = [ProjectId.new(), ProjectId.new()]
project_user_2_ids = [ProjectId.new(), ProjectId.new(), ProjectId.new()]

# projects owned by user 1
for i, project_id in enumerate(project_user_1_ids):
    project = (Project.new(project_id=project_id, user_id=user_id_1)
                .with_updated_metadata(
                    Metadata.new()
                    .with_updated_name(Name.from_str(f'Notebook Project 1_{i}'))
                    .with_updated_description(Description.from_str(f'This is project 1_{i} created in a notebook'))
                )
              )
    project_created_event = ProjectCreatedEvent.from_project(project, DateTime.now())

    project_event_bus.record(
        EventEnvelope(event=project_created_event, metadata=EventMetadata.without_creator({'created_in_notebook': True}))
    )
    
# projects owned by user 2
for i, project_id in enumerate(project_user_2_ids):
    project = (Project.new(project_id=project_id, user_id=user_id_2)
                .with_updated_metadata(
                    Metadata.new()
                    .with_updated_name(Name.from_str(f'Notebook Project 2_{i}'))
                    .with_updated_description(Description.from_str(f'This is project 2_{i} created in a notebook'))
                )
              )
    project_created_event = ProjectCreatedEvent.from_project(project, DateTime.now())

    project_event_bus.record(
        EventEnvelope(event=project_created_event, metadata=EventMetadata.without_creator({'created_in_notebook': True}))
    )
    
# public project
public_project_owner_user_id = UserId.new()
public_project_id = ProjectId.new()
public_project = (Project.new(project_id=public_project_id, user_id=public_project_owner_user_id)
                .with_updated_metadata(
                    Metadata.new()
                    .with_updated_name(Name.from_str(f'Public Project'))
                    .with_updated_description(Description.from_str(f'This is a public project created in a notebook'))
                )
                .with_updated_permissions(
                    Permissions.new(public_project_owner_user_id).with_updated_visibility(Visibility.PUBLIC)
                )
              )
project_created_event = ProjectCreatedEvent.from_project(public_project, DateTime.now())
project_event_bus.record(
    EventEnvelope(event=project_created_event, metadata=EventMetadata.without_creator({'created_in_notebook': True}))
)


## Read projects

In [2]:
from morpheus.project.application.read.ProjectReader import project_reader

another_user = UserId.new()
summaries = project_reader.get_project_summaries_with_role_for_user(another_user)
print('only public')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])

summaries = project_reader.get_project_summaries_with_role_for_user(user_id_1)
print('user 1')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])

summaries = project_reader.get_project_summaries_with_role_for_user(user_id_2)
print('user 2')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])


only public
[('Public Project', 'public', None)]
user 1
[('Notebook Project 1_0', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 1_1', 'private', <Role.OWNER: 'owner'>), ('Public Project', 'public', None)]
user 2
[('Notebook Project 2_0', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 2_1', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 2_2', 'private', <Role.OWNER: 'owner'>), ('Public Project', 'public', None)]


## Add members to project 1_1

In [3]:
from morpheus.project.types.Permissions import Role
from morpheus.project.domain.events.ProjectPermissionEvents.PermissionEvents import MemberAddedEvent

admin_id = UserId.new()
editor_id = UserId.new()
viewer_id = UserId.new()

project_id = project_user_1_ids[0]

admin_added_event = MemberAddedEvent.from_user_id_and_role(project_id, admin_id, Role.ADMIN, DateTime.now())
project_event_bus.record(EventEnvelope(event=admin_added_event, metadata=EventMetadata.without_creator({'created_in_notebook': True})))

editor_added_event = MemberAddedEvent.from_user_id_and_role(project_id, editor_id, Role.EDITOR, DateTime.now())
project_event_bus.record(EventEnvelope(event=editor_added_event, metadata=EventMetadata.without_creator({'created_in_notebook': True})))

viewer_added_event = MemberAddedEvent.from_user_id_and_role(project_id, viewer_id, Role.VIEWER, DateTime.now())
project_event_bus.record(EventEnvelope(event=viewer_added_event, metadata=EventMetadata.without_creator({'created_in_notebook': True})))
                         
                         

## Read projects

In [4]:
summaries = project_reader.get_project_summaries_with_role_for_user(admin_id)
print('admin')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])

summaries = project_reader.get_project_summaries_with_role_for_user(editor_id)
print('editor')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])

summaries = project_reader.get_project_summaries_with_role_for_user(viewer_id)
print('viewer')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])

summaries = project_reader.get_project_summaries_with_role_for_admin_user()
print('super admin (sees all projects as owner)')
print([(summary.project_name.to_str(), summary.visibility.to_str(), role) for (summary, role) in summaries])


admin
[('Notebook Project 1_0', 'private', <Role.ADMIN: 'admin'>), ('Public Project', 'public', None)]
editor
[('Notebook Project 1_0', 'private', <Role.EDITOR: 'editor'>), ('Public Project', 'public', None)]
viewer
[('Notebook Project 1_0', 'private', <Role.VIEWER: 'viewer'>), ('Public Project', 'public', None)]
super admin (sees all projects as owner)
[('Notebook Project 1_0', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 1_1', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 2_0', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 2_1', 'private', <Role.OWNER: 'owner'>), ('Notebook Project 2_2', 'private', <Role.OWNER: 'owner'>), ('Public Project', 'public', <Role.OWNER: 'owner'>)]
